# Initial codes

This notebook illustrates how Bayesian inference can be used to infer response rates of each group (basket) in a Basket trial.

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [3]:
import numpy as np
import pandas as pd
import arviz as az
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [4]:
from pyBasket.common import load_obj
from pyBasket.common import (
    GROUP_STATUS_EARLY_STOP_FUTILE,
    GROUP_STATUS_COMPLETED_INEFFECTIVE,
    GROUP_STATUS_COMPLETED_EFFECTIVE,
)

In [5]:
def get_output_filenames(result_dir, scenario, with_clustering_info, n_clusters):
    base_filename = f'scenario_{scenario}_clustering_{with_clustering_info}'
    if with_clustering_info:
        base_filename += f'_ncluster_{n_clusters}'

    out_trial_results = os.path.join(result_dir, base_filename + '_trial_results.p')
    return out_trial_results

In [6]:
out_dir = os.path.abspath(os.path.join('..', 'scripts', 'results'))
out_dir

'/Users/joewandy/Work/git/pyBasket/scripts/results'

In [7]:
scenarios = range(0, 6)
clustering = [False, True]
n_clusters = [5, 10]

In [8]:
def gather_data(scenarios, clustering, n_clusters, out_dir):
    data = {}

    for scenario in scenarios:
        for cl in clustering:
            nc_values = [None] if not cl else n_clusters

            for nc in nc_values:
                try:
                    fname = get_output_filenames(out_dir, scenario, cl, nc)
                    print(fname)                
                    trial_results = load_obj(fname)
                    key = (scenario, cl, nc)
                    data[key] = trial_results
                except FileNotFoundError:
                    pass
                
    return data

In [9]:
data = gather_data(scenarios, clustering, n_clusters, out_dir)

/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_5_trial_results.p


2023-06-27 22:48:04.132 | WARNING  | pyBasket.common:load_obj:82 - Old, invalid or missing pickle in /Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_10_trial_results.p. Please regenerate this file.


/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_5_trial_results.p


2023-06-27 22:48:05.432 | WARNING  | pyBasket.common:load_obj:82 - Old, invalid or missing pickle in /Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_10_trial_results.p. Please regenerate this file.


/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_3_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_3_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_3_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_4_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_4_clustering_True_ncluster_5_trial_results.p


2023-06-27 22:48:11.574 | WARNING  | pyBasket.common:load_obj:82 - Old, invalid or missing pickle in /Users/joewandy/Work/git/pyBasket/scripts/results/scenario_4_clustering_True_ncluster_10_trial_results.p. Please regenerate this file.


/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_4_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_5_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_5_clustering_True_ncluster_5_trial_results.p


2023-06-27 22:48:13.038 | WARNING  | pyBasket.common:load_obj:82 - Old, invalid or missing pickle in /Users/joewandy/Work/git/pyBasket/scripts/results/scenario_5_clustering_True_ncluster_10_trial_results.p. Please regenerate this file.


/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_5_clustering_True_ncluster_10_trial_results.p


In [10]:
def get_analysis(data, trial_idx, scenario=0, cl=False, nc=None, last_only=True):
    key = (scenario, cl, nc)
    analysis_results = {}

    if key not in data:
        print(f"No trial result found for key {key}")
        return

    trial_result = data[key]

    if trial_idx >= len(trial_result):
        print(f"Trial index {trial_idx} out of range.")
        return

    for analysis_name in trial_result[trial_idx].idfs:
        idfs = trial_result[trial_idx].idfs[analysis_name]
        if last_only and idfs:
            idfs = idfs[-1]
        analysis_results[analysis_name] = idfs

    return analysis_results

def get_analysis_by_name(analysis_results, analysis_name):
    if analysis_name not in analysis_results:
        print(f"No analysis results found for {analysis_name}")
        return

    idfs = analysis_results[analysis_name]

    return idfs


def display_analysis(idfs):
    if isinstance(idfs, list):
        for df in idfs:
            display(df)
    elif idfs is not None:
        display(idfs)
    else:
        print("No data to display.")


In [12]:
def get_true_arr(scenario):

    if scenario == 0:
        y_true = np.array([False, False, False, False, False, False])
    elif scenario == 1:
        y_true = np.array([True, False, False, False, False, False])
    elif scenario == 2:
        y_true = np.array([True, True, False, False, False, False])
    elif scenario == 3:
        y_true = np.array([True, True, True, False, False, False])
    elif scenario == 4:
        y_true = np.array([True, True, True, True, False, False])
    elif scenario == 5:
        y_true = np.array([True, True, True, True, True, False])
        
    y_true_all = []
    for trial_idx in range(total):
        y_true_all.append(y_true)
    y_true_all = np.array(y_true_all)
    return y_true_all

In [13]:
def get_pred_arr(data, total, analysis_name):
    y_pred_all = []
    for trial_idx in range(total):

        # get the data first
        analysis_results = get_analysis(data, trial_idx)

        df = get_analysis_by_name(analysis_results, analysis_name)
        df_status = df['group_status'] == GROUP_STATUS_COMPLETED_EFFECTIVE
        y_pred = df_status.values
        y_pred_all.append(y_pred)

    y_pred_all = np.array(y_pred_all)
    return y_pred_all

|                   | Predicted Negative | Predicted Positive |
|-------------------|--------------------|--------------------|
| Actual Negative   |        TN          |        FP          |
| Actual Positive   |        FN          |        TP          |

In [21]:
def get_percent_reject(y_true_all, y_pred_all, basket_idx):
    cm = confusion_matrix(y_true_all[:, basket_idx], y_pred_all[:, basket_idx])
    fp = cm[0, 1]
    return fp / total * 100

In [23]:
total = 500
n_scenario = 6
n_baskets = 6
analysis_names = ['independent', 'independent_bern', 'hierarchical_bern', 'BHM', 'pyBasket']

df_data = []
for analysis_name in analysis_names:
    for scenario_idx in range(n_scenario):
        y_true_all = get_true_arr(scenario_idx)
        y_pred_all = get_pred_arr(data, total, analysis_name)

        for basket_idx in range(n_baskets):
            reject = get_percent_reject(y_true_all, y_pred_all, basket_idx)
            row = [scenario_idx, analysis_name, basket_idx, reject]
            df_data.append(row)

reject_df = pd.DataFrame(df_data, columns=['scenario', 'analysis_name', 'basket_idx', 'reject'])
reject_df

,scenario,analysis_name,basket_idx,reject
0,0,independent,0,7.4
1,0,independent,1,9.8
2,0,independent,2,10.6
3,0,independent,3,9.8
4,0,independent,4,9.8
...,...,...,...,...
175,5,pyBasket,1,0.0
176,5,pyBasket,2,0.0
177,5,pyBasket,3,0.0
178,5,pyBasket,4,0.0


In [24]:
reshaped_df = reject_df.pivot(index=['scenario', 'analysis_name'], columns='basket_idx', values='reject')
reshaped_df

basket_idx                     0     1     2     3     4     5
scenario analysis_name                                        
0        BHM                11.8  12.0  11.4  11.6  11.2  12.6
         hierarchical_bern   7.6   9.8  11.2  10.0   9.6  10.6
         independent         7.4   9.8  10.6   9.8   9.8  11.0
         independent_bern    7.8  10.2  11.4  10.2  10.0  11.4
         pyBasket            7.8  10.0  11.2   9.4   9.0  11.2
1        BHM                 0.0  12.0  11.4  11.6  11.2  12.6
         hierarchical_bern   0.0   9.8  11.2  10.0   9.6  10.6
         independent         0.0   9.8  10.6   9.8   9.8  11.0
         independent_bern    0.0  10.2  11.4  10.2  10.0  11.4
         pyBasket            0.0  10.0  11.2   9.4   9.0  11.2
2        BHM                 0.0   0.0  11.4  11.6  11.2  12.6
         hierarchical_bern   0.0   0.0  11.2  10.0   9.6  10.6
         independent         0.0   0.0  10.6   9.8   9.8  11.0
         independent_bern    0.0   0.0  11.4  10.2  10.0  11.4
         pyBasket            0.0   0.0  11.2   9.4   9.0  11.2
3        BHM                 0.0   0.0   0.0  11.6  11.2  12.6
         hierarchical_bern   0.0   0.0   0.0  10.0   9.6  10.6
         independent         0.0   0.0   0.0   9.8   9.8  11.0
         independent_bern    0.0   0.0   0.0  10.2  10.0  11.4
         pyBasket            0.0   0.0   0.0   9.4   9.0  11.2
4        BHM                 0.0   0.0   0.0   0.0  11.2  12.6
         hierarchical_bern   0.0   0.0   0.0   0.0   9.6  10.6
         independent         0.0   0.0   0.0   0.0   9.8  11.0
         independent_bern    0.0   0.0   0.0   0.0  10.0  11.4
         pyBasket            0.0   0.0   0.0   0.0   9.0  11.2
5        BHM                 0.0   0.0   0.0   0.0   0.0  12.6
         hierarchical_bern   0.0   0.0   0.0   0.0   0.0  10.6
         independent         0.0   0.0   0.0   0.0   0.0  11.0
         independent_bern    0.0   0.0   0.0   0.0   0.0  11.4
         pyBasket            0.0   0.0   0.0   0.0   0.0  11.2